In [238]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from google.colab import drive

In [239]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [240]:
players21=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_21.csv')
players22=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_22.csv')

<ipython-input-240-a1e1a7d0c384>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players22=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_22.csv')


### data preparation & feature extraction process

In [241]:
players21.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [242]:
players21.shape

(18944, 110)

In [243]:
players21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


In [244]:
players21.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,18944.000000,18944.000000,18944.000000,1.870700e+04,18719.000000,18944.000000,18944.000000,18944.000000,18719.000000,18719.000000,...,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,2083.000000
mean,226232.328917,65.677787,71.086729,2.902288e+06,9148.482825,25.225823,181.190773,75.016892,47908.905551,1.355468,...,57.978674,46.470017,47.581767,45.546505,16.446052,16.236486,16.103357,16.225982,16.551309,37.203553
std,27166.574284,7.002278,6.109985,7.743775e+06,19893.149956,4.697354,6.825672,7.057140,53906.733776,0.739015,...,12.118390,20.169591,21.402461,20.953997,17.577332,16.845480,16.519399,17.017341,17.878121,10.714523
min,41.000000,47.000000,47.000000,9.000000e+03,500.000000,16.000000,155.000000,50.000000,1.000000,1.000000,...,12.000000,3.000000,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,210028.750000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,461.000000,1.000000,...,50.000000,29.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000,28.000000
50%,232301.500000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1913.000000,1.000000,...,59.000000,52.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000,37.000000
75%,246745.750000,70.000000,75.000000,2.000000e+06,8000.000000,29.000000,186.000000,80.000000,110986.000000,1.000000,...,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,45.000000
max,258970.000000,93.000000,95.000000,1.855000e+08,560000.000000,53.000000,206.000000,110.000000,114899.000000,4.000000,...,96.000000,94.000000,93.000000,90.000000,90.000000,92.000000,93.000000,91.000000,90.000000,65.000000


DROPPING NULL VALUES FOR 30 PERCENT

In [245]:

threshold = len(players21) * 0.3
players21_clean = players21.dropna(thresh=threshold, axis=1)
print("Shape of cleaned dataset:", players21_clean.shape)



Shape of cleaned dataset: (18944, 103)


In [246]:
players21.fillna(players21.mean(),inplace=True)

<ipython-input-246-ae12c5b4e547>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  players21.fillna(players21.mean(),inplace=True)


In [ ]:
correlation = players21.corr()
correlation

In [248]:
independent_features= players21[['overall']]

In [249]:
categorical_columns = players21.select_dtypes(exclude=['int', 'float']).columns.tolist()




In [250]:
encoder = OneHotEncoder(sparse=False, drop='first')
encoder.fit(players21[categorical_columns])
encoded_data = encoder.transform(players21[categorical_columns])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [251]:
correlation = players21.corr()
correlation_target = abs(correlation["overall"])
relevant_features = correlation_target[(correlation_target >= 0.5) & (correlation_target<1)]
relevant_features

<ipython-input-251-d2a22e18b272>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = players21.corr()


potential                  0.636366
value_eur                  0.555192
wage_eur                   0.589060
release_clause_eur         0.614344
passing                    0.662090
dribbling                  0.596558
attacking_short_passing    0.502191
movement_reactions         0.867234
power_shot_power           0.558372
mentality_vision           0.509087
mentality_composure        0.705252
Name: overall, dtype: float64

In [252]:
# Relevant features based on correlation analysis
relevant_features = [
    'potential', 'value_eur', 'wage_eur', 'release_clause_eur',
    'passing', 'dribbling', 'attacking_short_passing',
    'movement_reactions', 'power_shot_power', 'mentality_vision',
    'mentality_composure'
]

In [253]:
Select_relevant_features= players21[relevant_features]
Select_relevant_features

,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,93,103500000.0,560000.0,138400000.0,91.0,95.0,91,94,86,95,96
1,92,63000000.0,220000.0,75900000.0,81.0,89.0,82,95,94,82,95
2,91,111000000.0,240000.0,132000000.0,78.0,85.0,84,93,89,79,88
3,91,132000000.0,270000.0,166500000.0,86.0,94.0,87,91,80,90,93
4,91,129000000.0,370000.0,161000000.0,93.0,88.0,94,91,91,94,91
...,...,...,...,...,...,...,...,...,...,...,...
18939,52,70000.0,1000.0,57000.0,26.0,27.0,31,48,35,25,40
18940,53,70000.0,1000.0,72000.0,49.0,47.0,55,50,38,53,35
18941,47,45000.0,2000.0,47000.0,49.0,46.0,56,44,49,44,35
18942,67,130000.0,500.0,165000.0,40.0,53.0,45,53,50,49,45


In [255]:
scaler_standard = StandardScaler()
scaler_standard.fit(players21[relevant_features])
players21_features = scaler_standard.transform(players21[relevant_features])




In [256]:
with open('scaler.pkl', 'wb') as model_file:
    pickle.dump(scaler_standard, model_file)


In [ ]:
X = players21[relevant_features]
y = players21['overall']

In [ ]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (18944, 11)
Shape of y: (18944,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


MODEL TESTING

In [ ]:
 # Train Individual Models
rf = RandomForestRegressor(n_estimators=100, random_state=42)
gb = GradientBoostingRegressor(n_estimators=100)
lr = LinearRegression()

rf.fit(X_train, y_train)
gb.fit(X_train, y_train)
lr.fit(X_train, y_train)

# Soft Voting Ensemble
ensemble = VotingRegressor(estimators=[
    ('rf', rf),
    ('gb', gb),
    ('lr', lr)
])
ensemble.fit(X_train, y_train)

# Predict and Evaluate
y_ensemble_pred = ensemble.predict(X_test)
mae = mean_absolute_error(y_test, y_ensemble_pred)
print(f"Soft Voting Ensemble MAE: {mae}")

Soft Voting Ensemble MAE: 1.1204416798550292


In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
rf_mae = mean_absolute_error(rf_predictions, y_test)
print("RandomForestRegressor MAE:", rf_mae)
print("RandomForestRegressor RMSE:", np.sqrt(mean_squared_error(rf_predictions, y_test)))


RandomForestRegressor MAE: 0.6162549485352337
RandomForestRegressor RMSE: 1.074562190273496


In [ ]:
gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb.fit(X_train, y_train)
gb_predictions = gb.predict(X_test)
gb_mae = mean_absolute_error(gb_predictions, y_test)

print("GradientBoostingRegressor MAE:", gb_mae)
print("GradientBoostingRegressor RMSE:", np.sqrt(mean_squared_error(gb_predictions, y_test)))


GradientBoostingRegressor MAE: 1.1134804362253377
GradientBoostingRegressor RMSE: 1.5118541329517803


In [ ]:
xgb = XGBRegressor(n_estimators=100, random_state=42)
xgb.fit(X_train, y_train)

# Evaluate the model
y_pred = xgb.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("XGBoost MAE:", mae)
print("XGBoost RMSE:", rmse)

# Save the XGBoost model
with open('xgboost_model.pkl', 'wb') as model_file:
    pickle.dump(xgb, model_file)


XGBoost MAE: 0.7388328830117945
XGBoost RMSE: 1.0776779376882895


In [ ]:
decision_tree = DecisionTreeRegressor(random_state=42)
decision_tree.fit(X_train, y_train)

# Evaluate the model
y_pred = decision_tree.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Decision Tree MAE:", mae)
print("Decision Tree RMSE:", rmse)


Decision Tree MAE: 0.7550804961731328
Decision Tree RMSE: 1.5886739533525127


In [ ]:
cv=KFold(n_splits=3)


FINE TUNING THE BEST MODEL(RANDOM FOREST REGRESSOR)

In [217]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)


grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_


best_estimator.fit(X_train, y_train)


y_pred_tuned = best_estimator.predict(X_test)
mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))

print("Tuned RandomForestRegressor MAE:", mae_tuned)
print("Tuned RandomForestRegressor RMSE:", rmse_tuned)
print("Best Parameters:", best_params)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Tuned RandomForestRegressor MAE: 0.6144224509545175
Tuned RandomForestRegressor RMSE: 1.073189581906773
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}


DATA CLEANING OF 22 PLAYERS

In [218]:
# Define the threshold for dropping columns (30% missing values)
threshold = len(players22) * 0.3

# Drop columns with more than 30% missing values
players22_clean = players22.dropna(thresh=threshold, axis=1)

# Verify the shape of the cleaned dataset
print("Shape of cleaned dataset:", players22_clean.shape)


Shape of cleaned dataset: (19239, 103)


In [219]:

players22.fillna(players22.mean(), inplace=True)



<ipython-input-219-e5712de3a105>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  players22.fillna(players22.mean(), inplace=True)


In [220]:
interest_features = [
    'potential', 'value_eur', 'wage_eur', 'release_clause_eur',
    'passing', 'dribbling', 'attacking_short_passing',
    'movement_reactions', 'power_shot_power', 'mentality_vision',
    'mentality_composure'
]
Select_interest_features = players22[interest_features]
# Standardize the features
scaler_standard = StandardScaler()
players22_features = scaler_standard.fit_transform(players22[interest_features])
players22_features


array([[ 3.60177985,  9.88960128, 15.9980219 , ...,  2.13943257,
         3.00622837,  3.13102536],
       [ 3.43746979, 15.35095752, 13.42584387, ...,  2.44264937,
         1.98059683,  2.47307708],
       [ 3.27315974,  5.54683609, 13.42584387, ...,  2.74586617,
         1.61429986,  3.04878183],
       ...,
       [-2.64200212, -0.36195656, -0.4381957 , ..., -0.58951861,
        -0.58348201, -1.80358675],
       [-1.82045186, -0.36064057, -0.4381957 , ..., -0.74112701,
        -1.02303839, -0.89890786],
       [-1.82045186, -0.36064057, -0.4381957 , ..., -0.58951861,
        -0.58348201, -1.80358675]])

In [221]:
# Categorical columns (if any)
categorical_columns2 = players22.select_dtypes(exclude=['int', 'float']).columns.tolist()

In [222]:

# Use OneHotEncoder to encode categorical variables
encoder = OneHotEncoder(sparse=False,drop='first')
encoder.fit(players22[categorical_columns2])
encoded_data = encoder.transform(players22[categorical_columns2])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns2))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [223]:
# Correlation analysis
correlation = players22.corr()
correlation_target = abs(correlation["overall"])
interest_features = correlation_target[(correlation_target >= 0.5) & (correlation_target < 1)]
interest_features

<ipython-input-223-6f614b1ef1e5>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = players22.corr()


potential                  0.644275
value_eur                  0.553823
wage_eur                   0.601001
release_clause_eur         0.532528
passing                    0.663519
dribbling                  0.618419
attacking_short_passing    0.516788
movement_reactions         0.871823
power_shot_power           0.561180
mentality_vision           0.523025
mentality_composure        0.708867
Name: overall, dtype: float64

In [224]:
players22.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


TEST ON 22 PLAYERS

In [225]:
#Use the trained model to make predictions on players22 data
players22_predictions = grid_search.best_estimator_.predict(X_test)

# Calculate MAE and RMSE for the predictions
mae_players22 = mean_absolute_error(players22_predictions, y_test)
rmse_players22 = np.sqrt(mean_squared_error(players22_predictions, y_test))

print("RandomForestRegressor MAE on players22 data:", mae_players22)
print("RandomForestRegressor RMSE on players22 data:", rmse_players22)

RandomForestRegressor MAE on players22 data: 0.6144224509545175
RandomForestRegressor RMSE on players22 data: 1.073189581906773


In [259]:

with open('Fifa_prediction.pkl', 'wb') as model_file:
    pickle.dump(best_estimator, model_file)


In [258]:
best_estimator.predict(pd.DataFrame(X_train.iloc[8]).transpose())

array([74.01333333])